Name: Divya Vhawale

StudentId:16102243

Course:MSc in Data Analytics

Group no.:A

Project Title:Stress Detection Over Social Interaction Using Machine Learning

In [1]:
!pip install hvplot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 10.8 MB/s eta 0:00:00


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import pickle
import sklearn
import holoviews as hv
import hvplot.pandas
import nltk
from bokeh.io import output_notebook
output_notebook()
from hvplot import pandas
from pathlib import Path

#hv.extension("bokeh")
from sklearn.model_selection import cross_val_score


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC


In [4]:
# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/content/stress_detection.csv'): #treating a CSV file as a directory path.
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
df = pd.read_csv("/content/stress_detection.csv")
df.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,...,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,...,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,...,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,...,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,...,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


The code snippet calculates and displays the distribution of class labels in the "label" column of the DataFrame "df" using the value_counts() function.

In [6]:
class_distribution = df['label'].value_counts()
print(class_distribution)


1    1488
0    1350
Name: label, dtype: int64


In [7]:
df.shape,df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838 entries, 0 to 2837
Columns: 116 entries, subreddit to sentiment
dtypes: float64(106), int64(6), object(4)
memory usage: 2.5+ MB


((2838, 116), None)

Preprocesses text by removing HTML tags, non-alphabetic characters, and single-letter words, then converts multiple whitespaces to single spaces.
Tokenizes the text into words, converts them to lowercase, and filters out stopwords.
Stems the remaining words using the SnowballStemmer.
Note: This code prepares text data for various purposes, such as text embedding or TF-IDF analysis, by cleaning and transforming it into a format suitable for downstream tasks like machine learning or natural language processing.

In [8]:
nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english"))


def clean_text(text, for_embedding=False):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


It uses the clean_text function to preprocess the text data in the DataFrame column "text_clean", removing HTML tags, non-alphabetic characters, single-letter words, and stopwords, while also stemming the words.
This operation is performed on text data that has a length greater than 20 characters.
Note: The code snippet improves the quality of text data in the "text_clean" column of the DataFrame by applying the preprocessing steps defined in the clean_text function. The %%time magic command is used to measure the execution time of this process.

In [9]:
%%time
# Clean text
df["text_clean"] = df.loc[df["text"].str.len() > 20, "text"]
df["text_clean"] = df["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)

CPU times: user 2.85 s, sys: 12.5 ms, total: 2.87 s
Wall time: 3.01 s


It creates a binary column "grade_bad" in the DataFrame, where values 1-2 or 5-6 are labeled as "good" (0), and the rest are marked as "bad" (1).
The code filters out rows where the "text_clean" column is empty or contains "null" values and removes rows with missing values in the columns "grade_bad", "text", and "text_clean".
Note: The code snippet creates a binary classification target based on the original "label" column and then filters and cleans the DataFrame to prepare it for further analysis or modeling.

In [10]:
# Create binary grade, class 1-2 or 5-6  = good or bad
df["grade_bad"] = 0
df.loc[df["label"] >= 3, "grade_bad"] = np.NaN
df.loc[df["label"] >= 5, "grade_bad"] = 1

# Drop when any of x missing
df = df[(df["text_clean"] != "") & (df["text_clean"] != "null")]

df = df.dropna(
    axis="index", subset=["grade_bad", "text", "text_clean"]
).reset_index(drop=True)

The code snippet copies the DataFrame "df" into a new DataFrame named "df_clean" without altering the original DataFrame structure, and the comment indicates an intention to save the "df_clean" DataFrame to a CSV file. Additionally, the comment suggests loading data from a CSV file located at "../../data/processed/comments_clean.csv".

In [11]:
# data.to_csv("../../data/processed/comments_clean.csv", index=False)
df_clean = df.copy()
# data_clean = pd.read_csv("../../data/processed/comments_clean.csv")

In [12]:
df_clean.head(3)

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment,text_clean,grade_bad
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,...,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742,said felt way sugget go rest trigger ahead you...,0.0
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,...,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857,hey assist sure right place post goe current s...,0.0
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,...,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894,mom hit newspap shock would know like play hit...,0.0


The code snippet utilizes the Bokeh library to visualize the word frequency of the most common words in the "text_clean" column of the "df_clean" DataFrame. It creates a bar plot using hvplot with the 40 most common words, displaying their frequencies on the y-axis, and adjusting the plot size and y-axis formatting.

In [13]:
from bokeh.models import NumeralTickFormatter
# Word Frequency of most common words
word_freq = pd.Series(" ".join(df_clean["text_clean"]).split()).value_counts()
word_freq[1:40].rename("Word frequency of most common words in text").hvplot.bar(
    rot=45
).opts(width=700, height=400, yformatter=NumeralTickFormatter(format="0,0"))

:Bars   [index]   (Word frequency of most common words in text)

The code snippet uses hvplot to create a table visualization displaying the 10 least frequent words along with their corresponding frequencies from the "word_freq" Series.

In [14]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq").hvplot.table()

:Table   [index,freq]

The code snippet generates a bar plot using hvplot to visualize the distribution of ratings in the "label" column of the "df_clean" DataFrame, displaying the normalized counts for each rating category.

In [15]:
# Distribution of ratings
df_clean["label"].value_counts(normalize=True).sort_index().hvplot.bar(
    title="Distribution of label"
)

:Bars   [index]   (label)

The code snippet creates a TfidfVectorizer named "vectorizer" with specific parameters for generating unique word vectors from the "text_clean" column of the "df_clean" DataFrame. It considers word pairs (bigrams) within a certain frequency range while excluding very uncommon words (appearing in less than 10 observations) and very common words (appearing in 30% or more of the observations).

In [16]:
"""
Compute unique word vector with frequencies
exclude very uncommon (<10 obsv.) and common (>=30%) words
use pairs of two words (ngram)
"""
vectorizer = TfidfVectorizer(
    analyzer="word", max_df=0.3, min_df=10, ngram_range=(1, 2), norm="l2"
)
vectorizer.fit(df_clean["text_clean"])

TfidfVectorizer(max_df=0.3, min_df=10, ngram_range=(1, 2))

The code snippet extracts a sample of five unique word (ngram) vectors from the vocabulary created by the "vectorizer" and displays them using a Pandas Series.

In [17]:
# Vector representation of vocabulary
word_vector = pd.Series(vectorizer.vocabulary_).sample(5, random_state=1)
print(f"Unique word (ngram) vector extract:\n\n {word_vector}")

Unique word (ngram) vector extract:

 reddit       1265
view         1684
interview     799
promis       1196
privat       1187
dtype: int64


The code snippet splits the "df_clean" DataFrame into a training set ("train") and a test set ("test") using a 75%-25% ratio. It separates the text data into "X_train" and "X_test", and the corresponding labels into "Y_train" and "Y_test". The shapes of the training and test sets are displayed using print statements.

In [18]:
# Sample data - 25% of data to test set
train, test = train_test_split(df_clean, random_state=1, test_size=0.25, shuffle=True)

X_train = train["text_clean"]
Y_train = train["label"]
X_test = test["text_clean"]
Y_test = test["label"]
print(X_train.shape)
print(X_test.shape)

(2125,)
(709,)


The code snippet transforms the text data in the "X_train" and "X_test" sets into numeric vectors using the TF-IDF values as elements. It displays the shape of the transformed "X_train_vec" sparse matrix using the get_shape() method.

In [19]:
# transform each sentence to numeric vector with tf-idf value as elements
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)
X_train_vec.get_shape()

(2125, 1801)

The code snippet compares the original comment text from index 3 in the training set ("X_train") with its corresponding numeric vector representation. It then creates a DataFrame "features" that holds the feature matrix representation of the text, filtering out columns with all zero values. Finally, it displays the non-empty vector representation of the sentence.

In [20]:
# Compare original comment text with its numeric vector representation
print(f"Original sentence:\n{X_train[3:4].values}\n")
# Feature Matrix
features = pd.DataFrame(
    X_train_vec[3:4].toarray(),
    columns=vectorizer.get_feature_names_out()
    )
nonempty_feat = features.loc[:, (features != 0).any(axis=0)]
print(f"Vector representation of sentence:\n {nonempty_feat}")

Original sentence:
['poor choic exacerb situat vase game control tv remot coffe mug etc thrown vari encount exagger list tri leav block door bodi threaten kill one occass']

Vector representation of sentence:
      block      bodi     choic     coffe   control      door       etc  \
0  0.26545  0.228095  0.275786  0.303477  0.212747  0.219788  0.201704   

    exagger      game      kill      leav     list      one      poor  \
0  0.289111  0.246056  0.221192  0.175583  0.24856  0.12623  0.273526   

     situat  threaten       tri        tv  
0  0.187338  0.247291  0.130352  0.292307  


The code snippet defines a list of classifiers including Logistic Regression, Linear Support Vector Classifier (LinearSVC), Random Forest, XGBoost, and Multi-Layer Perceptron (MLP) classifiers. It extracts the names of these classifiers using regular expressions and displays the list of classifier names to be tested.

In [21]:
# models to test
classifiers = [
    LogisticRegression(solver="sag", random_state=1),
    LinearSVC(random_state=1),
    RandomForestClassifier(random_state=1),
    XGBClassifier(random_state=1),
    MLPClassifier(
        random_state=1,
        solver="adam",
        hidden_layer_sizes=(12, 12, 12),
        activation="relu",
        early_stopping=True,
        n_iter_no_change=1,
    ),

]
# get names of the objects in list (too lazy for c&p...)
names = [re.match(r"[^\(]+", name.__str__())[0] for name in classifiers]
print(f"Classifiers to test: {names}")

Classifiers to test: ['LogisticRegression', 'LinearSVC', 'RandomForestClassifier', 'XGBClassifier', 'MLPClassifier']


The code snippet measures the execution time using %%time and iterates through a list of classifiers, training each one on the transformed training data ("X_train_vec" and "Y_train"). It then predicts labels on the transformed test data ("X_test_vec") and generates a classification report using sklearn.metrics. The results, including precision, recall, and F1-score metrics, are saved in the "results" dictionary.

In [22]:
%%time
# test all classifiers and save pred. results on test data
results = {}
for name, clf in zip(names, classifiers):
    print(f"Training classifier: {name}")
    clf.fit(X_train_vec, Y_train)
    prediction = clf.predict(X_test_vec)
    report = sklearn.metrics.classification_report(Y_test, prediction)
    results[name] = report

Training classifier: LogisticRegression
Training classifier: LinearSVC
Training classifier: RandomForestClassifier
Training classifier: XGBClassifier
Training classifier: MLPClassifier
CPU times: user 16.1 s, sys: 164 ms, total: 16.3 s
Wall time: 20.7 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The code snippet iterates through the dictionary "results" and displays the prediction results for each key (represented by "k") along with their corresponding values (represented by "v").

In [23]:
# Prediction results
for k, v in results.items():
    print(f"Results for {k}:")
    print(f"{v}\n")

Results for LogisticRegression:
              precision    recall  f1-score   support

           0       0.74      0.71      0.73       336
           1       0.75      0.77      0.76       373

    accuracy                           0.74       709
   macro avg       0.74      0.74      0.74       709
weighted avg       0.74      0.74      0.74       709


Results for LinearSVC:
              precision    recall  f1-score   support

           0       0.72      0.68      0.70       336
           1       0.73      0.76      0.74       373

    accuracy                           0.72       709
   macro avg       0.72      0.72      0.72       709
weighted avg       0.72      0.72      0.72       709


Results for RandomForestClassifier:
              precision    recall  f1-score   support

           0       0.69      0.64      0.66       336
           1       0.70      0.74      0.72       373

    accuracy                           0.69       709
   macro avg       0.69      0.69  

The code snippet creates a pipeline "pipe" that combines the TfidfVectorizer and LinearSVC components. It defines a parameter space for hyperparameter tuning, uses GridSearchCV to optimize the pipeline, fits the model on the training data, and saves the trained model using pickle. The execution time of this process is measured using the runtime comment.

In [24]:
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", LinearSVC())])

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 1), (1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8, 0.2),
    "tfidf__min_df": np.arange(5, 100, 45),
}
pipe_clf = GridSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro")
pipe_clf.fit(X_train, Y_train)
pickle.dump(pipe_clf, open("./clf_pipe.pck", "wb"))


In [25]:
print(pipe_clf.best_params_)

{'tfidf__max_df': 0.3, 'tfidf__min_df': 50, 'tfidf__ngram_range': (1, 1)}


The code snippet performs feature creation and modeling using the Pipeline with TfidfVectorizer and LinearSVC. It defines a parameter space for hyperparameter tuning, utilizes GridSearchCV for optimization, fits the model on the training data, and saves the trained model using pickle. The execution time of this process is measured using the %%time magic command.

In [26]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", LinearSVC())])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 3)],
    "tfidf__max_df": [0.5],
    "tfidf__min_df": [5],
    "svc__C": np.arange(0.2, 1, 0.15),
}
pipe_svc_clf = GridSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro")
pipe_svc_clf.fit(X_train, Y_train)
pickle.dump(pipe_svc_clf, open("./pipe_svc_clf.pck", "wb"))

CPU times: user 675 ms, sys: 68.1 ms, total: 743 ms
Wall time: 11.5 s


The code snippet prints the best parameters obtained from hyperparameter tuning using the best_params_ attribute of the fitted pipe_svc_clf model.






In [27]:
best_params = pipe_svc_clf.best_params_
print(best_params)

{'svc__C': 0.2, 'tfidf__max_df': 0.5, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 3)}


The code snippet fits the pipeline "pipe" with optimized parameters on the training data and then uses it to predict labels on the test data. It generates a classification report using sklearn.metrics and displays the report, including precision, recall, and F1-score metrics.

In [28]:
# run pipe with optimized parameters
pipe.set_params(**best_params).fit(X_train, Y_train)
pipe_pred = pipe.predict(X_test)
report = sklearn.metrics.classification_report(Y_test, pipe_pred)
print(report)

              precision    recall  f1-score   support

           0       0.74      0.71      0.73       336
           1       0.75      0.78      0.76       373

    accuracy                           0.75       709
   macro avg       0.75      0.75      0.75       709
weighted avg       0.75      0.75      0.75       709



The code calculates the confidence scores using the decision function of the trained pipeline "pipe" on the test data. It then determines the thresholds for negative and positive score based on the Nth highest and lowest scores, respectively, and displays these thresholds.

In [29]:
# Get confidence score for prediction
conf_score = pipe.decision_function(X_test)
# Get the Nth highest / lowest score
# high score indicates class 1 (bad), low score 0 (good)
score_neg = np.sort(conf_score)[-400]
score_pos = np.sort(conf_score)[708]
print(
    f"Threshold for negative score: {score_neg}\nThreshold for positive score: {score_pos}"
)



Threshold for negative score: -0.019418558499835814
Threshold for positive score: 1.5180174901786647


The code snippet adjusts the display settings to allow for wider columns and then filters the "test" DataFrame to show rows where the predicted labels ("pipe_pred") differ from the ground truth labels ("Y_test") and the confidence scores ("conf_score") are less than or equal to a specified positive threshold ("score_pos"). It displays the actual labels and text samples for instances where the model predicted "stress" despite the ground truth label indicating "not_stress".

In [30]:
pd.options.display.max_colwidth = 800
# Predicted not_stress but rated stress
test[["label", "text"]][(Y_test != pipe_pred) & (conf_score <= score_pos)]

,label,text
218,0,"One anxiety free day. I feel like I could live off that high for the rest of my life. Not starting my day off trying to catch my breath. Not having to plan out every single minute of my day as to avoid discomfort, or even panic. Not standing in the middle of a locker room getting ready to cry because you saw a lock on the locker you use every time you're at the gym."
1201,1,"I met my current boyfriend (42M), let’s call him V, back in November. Things were fine, there were no red flags. We dated about a month before one day I went to warm up his coffee for him and he started screaming at me for doing it wrong, and to never do it again. I have PTSD from being raped and abused, and I get uncomfortable when guys yell in the vicinity of me, when they yell at me, it’s debilitating. He knew this as I make it very clear to potential significant others."
2067,1,"I need some advice here. I'm not sure if my insecurities have run wild or if I'm justified in thinking my boyfriend is being out of line. He met this girl through friends who set them up and for their first date they went jogging. Weird, I know, but it's her hobby and he wanted to get into it. They'd never met before this."
2810,1,"I couldn't speak but finally managed to say, ""Bag, get my bag."" He was able to help me with my pills and touch me and talk to me enough that I came back. Now boyfriend is back to bed, and I'm assuming I'll be up for the rest of the night. I don't know what to do with this. This is scary stuff."
1922,1,"Mostly, I think I just want a mom. I've been struggling with anxiety really bad, and a lot of it has to do with my relationship with my boyfriend. This is my first boyfriend. He's a senior at a different college. He lives in the city that I go to school in."
...,...,...
1836,0,"Her name was Chinook. I wasn't there. Edit--the vet suspected she had a bad heart. We had to move to find work, back in our home province of Ontario. I'm working (moved suddenly because I had an interview at a job I knew I would get), and he's already done an interview, despite not being here for more than a few days."
1761,1,"And I woke up. It took 3 nurses and some sort of shot to calm me down when I woke up and I had to lay there limp for an hour in the recovery room by myself. They then put me in the maternity ward, sharing a room with a mother who just had her baby. The second surgery was a few days later because the surgery was incomplete. I found out when the part of the baby came out and I slipped on it in the bathroom, my husband found me in a pool of blood crying on the ground."
288,0,Well I guess it was my second. Last week we set the ground work for the safe space to come back to if things got too overwhelming. This week we began to work through my trauma. Starting out the session was definitely very strange. I wasn’t sure what answers my therapist was looking for but once she assured me that there were no wrong answers everything went fine.
1687,0,I know it is a huge accomplishment that I've been able to get back on the road and try again. It's just frustrating that I haven't been making as much progress as I'd like. I really miss being independent and free. Does anyone have any tips? I'd love to find someone that can relate or who has overcome this fear.


The code snippet filters the "test" DataFrame to display rows where the predicted labels ("pipe_pred") do not match the ground truth labels ("Y_test") and the confidence scores ("conf_score") are greater than or equal to a specified negative threshold ("score_neg"). It shows the actual labels and corresponding text samples for instances where the model predicted "not_stress" when the ground truth label indicated "bad stress".

In [31]:
# Predicted bad stress rated not_stress
test[["label", "text"]][(Y_test != pipe_pred) & ((conf_score >= score_neg))]

,label,text
218,0,"One anxiety free day. I feel like I could live off that high for the rest of my life. Not starting my day off trying to catch my breath. Not having to plan out every single minute of my day as to avoid discomfort, or even panic. Not standing in the middle of a locker room getting ready to cry because you saw a lock on the locker you use every time you're at the gym."
2006,0,"one day i broke down into absolute tears and was rushed into the counselers office. at this point im in 8th grade and this is by no means acceptable behavior. we are about to go into high school and i just burst into a ball of tears in front of every last one of my peers. the good thing is, this scenario made me care a little less about what everyone thought of me due to the already absolute embarrassment of myself. My mindset, the people i surrounded myself with, and my family life were all fucked to say the least... especially family."
1392,0,"So I'm getting a cheap apartment, most utilities included, and all maintenance covered. I used the last of my savings for the application fee and hold fee. I'm going to be moving at the end of the month and it's going to be tight, paying the security deposit and prorated rent when I already had to pay lot rent for the trailer this month. I'm selling the trailer at a loss to get about a third of it, selling everything I don't need in the hopes for another third, and I should be able to cover the rest from my paycheck... If I cut the grocery bill."
866,0,"Also, to answer the million dollar question everyone has, ""How does this happen to someone who outearns her abuser by a factor of 6?"" I wish people talked more about the other side of financial abuse. We hear all about the men who financially abuse their partners by prohibiting them from having a job, or denying them access to bank accounts, and that sort of thing. There's also the side where they monitor every account and become completely terrifying if their higher earning partner so much as buys one cup of coffee that they didn't approve of. They spend all her money on random crap that they can pass off as household expenses."
2732,0,(One is 3 yrs old and the other 19) . Now my dad would beat my mom but over the course of the years as I grew older he turned the violence onto me. He would see that I defended my mom and since I called the police one time (I found him choking her onto the wall) he stopped hurting her and transitioned into beating me. Now I know you guys may ask why did you continue to allow the abuse with you if you had stopped it with your mother before? Well I have an answer to that my mother loved my father very much and every time I tried contacting the police she would stop me and I have a lot of love towards my mother and would do anything for her... but anyways back to the present we no longer live with him anymore and we have cut all ties with him except for my brother.
...,...,...
1592,0,"If this is all I'm going to experience side-effect wise, then I regret not starting earlier. I really hope this will help with the anxiety, panic disorder, and depression. What are your experiences with antidepressants? Does it get worse? I just can't wait to start actually feeling like myself again."
1836,0,"Her name was Chinook. I wasn't there. Edit--the vet suspected she had a bad heart. We had to move to find work, back in our home province of Ontario. I'm working (moved suddenly because I had an interview at a job I knew I would get), and he's already done an interview, despite not being here for more than a few days."
288,0,Well I guess it was my second. Last week we set the ground work for the safe space to come back to if things got too overwhelming. This week we began to work through my trauma. Starting out the session was definitely very strange. I wasn’t sure what answers my therapist was looking for but once she assured me that there were no wrong answers everything went fine.
1687,0,I know it is a huge accomplishment that 

The provided code defines a list named "INPUT" containing two text samples, each describing contrasting emotional experiences: one negative and stressful, and the other positive and uplifting.

In [32]:
INPUT = [
    "This is the worst thing that happened to me today. I got less marks in my exam,so it is not going to help me in my future.",
    "Hi Shashank sir, I gained a lot of knowledge from you for my future use.This was a very fun journey for me. Thanks for boosting my confidence."
]

The code snippet preprocesses the comments in the "INPUT" list using the clean_text function, then concatenates them into a formatted text block. It subsequently uses the trained pipeline "pipe" to predict labels for the preprocessed comments.

In [33]:
# Pre-Process comments as we did with train data
text = [clean_text(text) for text in INPUT]
text_out = " \n".join(text)
print(f"Input after pre-processing / cleaning:\n\n{text_out}")
# run comments through pipe: predict using our best model from above
predictions = pipe.predict(text)

Input after pre-processing / cleaning:

worst thing happen today got less mark exam go help futur 
hi shashank sir gain lot knowledg futur use fun journey thank boost confid


The code snippet combines the input text data (stored in "INPUT") and the predicted labels (converted from 0 and 1 to "not_stress" and "stress") into a DataFrame with columns "text" and "prediction" using pd.concat.

In [34]:
# Show comment and predicted Labels
predictions = pd.Series(predictions)
predictions = predictions.replace(0, "not_stress").replace(1, "stress")

pd.concat(
    [pd.Series(INPUT), predictions], axis="columns", keys=["text", "prediction"]
)

,text,prediction
0,"This is the worst thing that happened to me today. I got less marks in my exam,so it is not going to help me in my future.",stress
1,"Hi Shashank sir, I gained a lot of knowledge from you for my future use.This was a very fun journey for me. Thanks for boosting my confidence.",not_stress
